In [87]:
%pylab ipympl
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [88]:
data = pd.read_csv('/Users/jennifer/Desktop/School/Year 4/Astro 329/Lab 2/data.txt',delim_whitespace=True)
#data is just the bias subtracted data
data2 = pd.read_csv('/Users/jennifer/Desktop/School/Year 4/Astro 329/Lab 2/data2.txt',delim_whitespace=True)
#data2 is the data after flat fielding 
exposure = pd.read_csv('/Users/jennifer/Desktop/School/Year 4/Astro 329/Lab 2/exposure.txt',delim_whitespace=True)
#exposure times of each frame 

In [89]:
import scipy.stats as stats
from scipy import interpolate
from scipy import optimize

### Sigma vs  Signal Plot

In [90]:
close(1);figure(1)

plot(data['MEAN'],data['STD'],'o',label='Data 1')
plot(data2['MEAN'],data2['STDDEV'],'o',label='Data 2')


dat = data['STD']
low = where(dat < 20)

for i in low:
    plot(data['MEAN'][i],data['STD'][i],'o',color='orange')


xscale('log');yscale('log')
grid()
xlabel('Signal [DU]');ylabel('$\sigma$ [DU]')
legend()
title('$\sigma$ as a function of signal')
#vlines(30000,0,210);vlines(40000,0,210)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$\\sigma$ as a function of signal')

### Calculating alpha and alpha vs signal plot

In [91]:
Nff = sqrt((data['STD'])**2 - (data2['STDDEV'])**2)
allalpha = Nff / (data2['MEAN'])

good_alpha = []
for i in allalpha:
    
    if i < 0.004:
        #
        if i > 0.002:
            good_alpha.append(i)
            
alpha = mean(good_alpha)
print('The alpha value is {}'.format(alpha))

The alpha value is 0.0033163996432357336


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [179]:
Nff

0           NaN
1      1.941520
2      2.461869
3      7.368392
4      4.621688
        ...    
56    18.799415
57    24.431146
58    27.344060
59    33.761214
60    37.837250
Length: 61, dtype: float64

In [102]:
close(6);figure(6)
plot(data2['MEAN'], Nff/(data2['MEAN']) ,'o')

xlabel('Signal [DU]')
ylabel('\u03B1')
title('\u03B1 as a function of signal')
grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Signal vs exposure time plot

In [7]:
exp = exposure['EXPOSURE']
close(1);figure(1)
plot(exposure['EXPOSURE'],data['MEAN'],'o')

ind = [where(exp==275)[0][0],where(exp==300)[0][0],where(exp==350)[0][0]]
for i in ind:
    plot(exposure['EXPOSURE'][i],data['MEAN'][i],'-or')
grid()
xlabel('Exposure Time');ylabel('Signal')
title('Signal as a function of exposure time')
hlines(30000,0,600);hlines(40000,0,600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Readout noise

In [79]:
biassub_std = pd.read_csv('/Users/jennifer/Desktop/School/Year 4/Astro 329/Lab 2/biassub_std.txt',delim_whitespace=True)
std = mean(biassub_std['STDDEV'])


sigma_RO = std/(sqrt(2))
print('The readout noise of a single readout is {}'.format(sigma_RO))

RO2 = (1 + 1/28)*(sigma_RO**2)
print('The readout noise of a flat frame is {}'.format(RO))
sqrt(RO2)

The readout noise of a single readout is 1.4628271535796697
The readout noise of a flat frame is 1.5150709804932294


1.488719909810462

In [80]:
RO_e = sqrt(RO2) * 3
RO_e

4.466159729431386

### Gain Method 1

In [75]:
def gain1( s , sigma ): 
    '''Method 1 of calculating gain
    '''
    g = s /( sigma**2 - RO2)
    return g

In [238]:
# First Guess at Gain - Dat 1 Method 1
g1_dat1 = gain1(data['MEAN'], data['STD']) 

g1_good =  []
for val in g1_dat1:
    if val > 0.0:
        if val < 10.0:
            g1_good.append(val)

print('The gain calculated with the bias-subtarctd images is {:.3f}'.format(mean(g1_good))) #This is the first guess at the CCD gain 

The gain calculated with the bias-subtarctd images is 2.519


In [241]:
# Second Guess at Gain - Dat 2 Method 1
g1_dat2 = gain1(data2['MEAN'],data2['STDDEV'])

g1_dat2_good =  []
for i,val in enumerate(g1_dat2):
    if data['MEAN'][i] < 20000.0:
        if val > 0:
            g1_dat2_good.append(val)

print('The gain calculated with the flat-fielded images is {:.3f}'.format(mean(g1_dat2_good)))

The gain calculated with the flat-fielded images is 2.914


### Gain Method 2

In [242]:
def gain2( s , sigma, nff):
    g = s / ( sigma**2 - RO2 - nff**2 )
    return g

In [243]:
g2_dat2 = gain2(data['MEAN'],data['STD'],Nff)


g2_dat2_good =  []
for i,val in enumerate(g2_dat2):
    if data['MEAN'][i] < 20000.0:
        if val > 0:
            g2_dat2_good.append(val)

print('The gain calculated with the bias-subtracted images is {:.3f}'.format(mean(g2_dat2_good)))

The gain calculated with the bias-subtracted images is 2.925


### Gain Method 3

In [125]:
def gain3( s , sigma , alpha):
    g = s / ( sigma**2 - RO2 - (alpha*s)**2 )
    return g

In [255]:
g3_dat1 = gain3(data['MEAN'],data['STD'],alpha)

g3_dat1_good =  []
for val in g3_dat1:
    if val > 0.0:
        if val < 5:
            g3_dat1_good.append(val)

print('The gain calculated with the flat-fielded images is {:.3f}'.format(mean(g3_dat1_good)))



g3_dat1_good2 =  []
for i,val in enumerate(g3_dat1):
    if data['MEAN'][i] < 1000000.0:
        if val > 0:
            g3_dat1_good2.append(val)
            
print('The gain calculated with the flat-fielded images is {:.3f}'.format(mean(g3_dat1_good2)))

The gain calculated with the flat-fielded images is 2.774
The gain calculated with the flat-fielded images is 8.032


In [254]:
print(g3_dat1)

0    -1.864779
1     3.640686
2     3.164589
3     2.480492
4     3.149820
        ...   
56    2.639145
57    2.759560
58    2.813707
59    2.605864
60    2.474030
Length: 61, dtype: float64


### Gain Plot

In [281]:
close(5);figure(5)
plot(data['MEAN'], gain1(data['MEAN'], data['STD']) ,'o',color='red',marker='*',label='non-flat fielded data - Method 1')


plot(data2['MEAN'], gain1(data2['MEAN'], data2['STDDEV']) ,'o',color='blue',marker='s',label='flat fielded data - Method 1')

plot(data['MEAN'], gain2(data['MEAN'], data['STD'], Nff),'o',marker='v',color='orange',label='non-flat fielded data - Method 2')

plot(data['MEAN'], gain3(data['MEAN'], data['STD'],alpha),'o',marker='d',color='purple',label='non-flat fielded data - Method 3')

grid()
hlines(3,-2000,55000)
ylim(0,10)
legend()
title('Gain calculated using different methods as a function of signal')
xlabel('Signal [DU]');ylabel('Gain')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Gain')

In [282]:
50000 *3

150000

In [283]:
50000 * 2.93

146500.0

In [286]:
ls = [179.9, 150.6, 144.1, 140.8]
mean(ls)

153.85000000000002

In [287]:
(mean(ls) *3)/30

15.385000000000002

In [299]:
npix_bef = np.array([4194304,4194304,4194304,4194304])
npix_aft = np.array([4191106,4190818,4190833,4190832])
mean(abs(npix_aft - npix_bef))/30

113.55833333333334

In [297]:
npix_bef2 = 4188466,4187966,4187949,4187999
mean(abs(npix_aft - npix_bef2))/30

93.40833333333333